# **Knight Vision - YOLOv8 Model Training Notebook**
---

This notebook trains the official YOLOv8 model used for chess piece detection as part of the Knight Vision system.

### **Project Overview**

- **Subject:** Deep Learning and Convolutional Neural Network (42028)
- **Project:** Knight Vision - Chess Board & Piece Detection
- **Model:** YOLOv8 (Large variant - `yolov8l`)
- **Dataset:** Custom dataset (augmented and manually corrected), prepared and split into `train` and `val` directories
- **Configuration:** Predefined `data.yaml` file for dataset paths and class names
- **Training Resolution:** 640x640
- **Epochs:** 50

This notebook is intended to exactly replicate the training pipeline used for the final Knight Vision chess detector model and can be used to retrain or further tune the model easily from the prepackaged dataset and config.

<br>

### **Workflow Overview**

1. Load the dataset configuration (`data.yaml`) which points to the existing dataset and split folders.
2. Initialise the YOLOv8 model architecture (using `yolov8l.yaml` to train from scratch or `yolov8l.pt` to fine-tune).
3. Train the model for 50 epochs.
4. Validate after training.
5. Optionally test on new images after training (optional cell).


---
## **1. Initial Set Up**

#### **1.1. Install Dependencies**

In [ ]:
%pip install ultralytics
%pip install matplotlib
%pip install pillow
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### **1.2. Import Necessary Libaries/Packages**

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import os
import yaml
from tqdm import tqdm

#### **1.3. Load Dataset Configuration**

In [ ]:
dataset_dir = "./chess_yolo_dataset"

# Load the dataset config (data.yaml)
config_path = os.path.join(dataset_dir, "data.yaml")
with open(config_path, "r") as f:
    data_config = yaml.safe_load(f)

print("Successfully Loaded Dataset Config:")
print(data_config)

---
## **2. Initialise YOLOv8 Model**

In [ ]:
model = YOLO("yolov8l.pt")

---
## **3. Train the Model**

In [ ]:
model.train(
    data=config_path,
    epochs=100,
    imgsz=640,
    batch=12,
    patience=15,
    project="yolo_training_logs",
    name="yolov8l_chess",
    exist_ok=True
)

> **Notes:**
> - The model checkpoints and training logs will be saved under `knight_vision_training/yolov8l_chess/` folder.
> - We used imgsz=416 for consistency with the input size used during data preparation and inference (to match detection grid).

---
## **4. Visualise Training Results**

In [ ]:
# Load the Trained Model Results
results_folder = "./knight_vision_training/yolov8l_chess"
results_file = os.path.join(results_folder, "results.png")

# Show the Results
if os.path.exists(results_file):
    img = Image.open(results_file)
    plt.imshow(img)
    plt.axis('off')
    plt.title("Training Results")
    plt.show()
else:
    print("Results Image Not Found -> Training may still be running or is incomplete.")

---
## **5. Load Final Trained Model and Test (Sanity Check)**

In [ ]:
trained_model = YOLO(os.path.join(results_folder, "weights", "best.pt"))

test_img_path = "./test_board.jpg"

if os.path.exists(test_img_path):
    trained_model.predict(test_img_path, imgsz=416)
else:
    print("No Test Image Provided.")